In [1]:
import os
from pathlib import Path

import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"


current_dir = Path(os.getcwd())

PROJECT = current_dir.parent.resolve()
SRC = PROJECT.parent.resolve()
ROOT = PROJECT.joinpath("..", "..").resolve()

BLD = ROOT.joinpath("bld").resolve()

# Folders inside idos_ppp
DATA = PROJECT.joinpath("data").resolve()
DATA_MGT = PROJECT.joinpath("data_management").resolve()
ANALYSIS = PROJECT.joinpath("analysis").resolve()
FINAL = PROJECT.joinpath("final").resolve()

DOCUMENTS = ROOT.joinpath("documents").resolve()

print(ROOT)

/Users/sergeimolinari/Desktop/IDOS/IDOS-PPP


# TRY THE NEW DATASET FOR POSSIBLE ERRORS

In [2]:
from idos_ppp.config import DATA
from idos_ppp.parameters import sheet_names

data_file = DATA
raw_data_path=DATA / "Data_2007_2010_2013_2016_2019.xlsx"

# Open the dataset file using pandas
try:
    raw_dta = {
        sheet: pd.read_excel(raw_data_path, sheet_name=sheet, header=1)
        for sheet in sheet_names
    }
    print("Data loaded successfully.")
    # Display the first few rows of one of the dataframes
    for sheet, df in raw_dta.items():
        print(f"First few rows of {sheet}:")
        print(df.head())
except FileNotFoundError:
    print(f"Error: The file {raw_data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully.
First few rows of Transformed 2007:
    Unnamed: 0   Unnamed: 1 FFP X1   1-2 UCDP WEF GCI Subpillar: 5. Security  \
0  Afghanistan  Afghanistan       0         0                            XXX   
1      Albania      Albania    0.44         1                       0.691667   
2      Algeria      Algeria    0.43  0.458492                       0.768333   
3       Angola       Angola    0.24  0.899611                           0.86   
4    Argentina    Argentina    0.72         1                           0.69   

  1-4 Political Terror 1-5 FFP P3 Protection  \
0                    0       0.18      0.045   
1                 0.75       0.46   0.668333   
2                 0.25       0.26       0.43   
3                  0.5       0.25        0.5   
4                 0.75       0.63       0.72   

  WEF GCR 2nd pillar: Infrastructure 2-1-2 UN: Telecom (2008)   ...  \
0                                XXX                   0.01576  ...   
1                         

In [3]:
from idos_ppp.data_management.idos_datamanagement import clean_and_concatenate_data

def clean_and_concatenate(
    raw_data=DATA / "Data_2007_2010_2013_2016_2019.xlsx",
    produces=BLD / "data" / "clean_data_new.csv",
):
    raw = {
        year: pd.read_excel(raw_data, sheet_name=sheet, header=1)
        for year, sheet in zip(years, sheet_names, strict=False)
    }
    countries_list = pd.read_excel(raw_data, sheet_name="3-2", header=0)
    clean_data = clean_and_concatenate_data(
        raw_data_dict=raw, country_codes=countries_list.iloc[0:, 1].tolist()
    )
    clean_data.to_csv(produces)

In [4]:
from idos_ppp.config import DATA, BLD
from idos_ppp.data_management.idos_datamanagement import clean_and_concatenate_data
from idos_ppp.parameters import sheet_names, years

bld_file = BLD
    
# Construct the path to the files
raw_data_path=DATA / "Data_2007_2010_2013_2016_2019.xlsx"
clean_data_path = BLD / "data" / "clean_data_new.csv"

try:
    clean_data = clean_and_concatenate_data(
    raw_data_dict={
        year: pd.read_excel(raw_data_path, sheet_name=sheet, header=1)
        for year, sheet in zip(years, sheet_names, strict=False)
    },
    country_codes=pd.read_excel(raw_data_path, sheet_name="3-2", header=0).iloc[0:, 1].tolist()
    )
    clean_data.to_csv(clean_data_path)
    print("Data loaded successfully.")
    # Display the first few rows of the dataframe
    print(clean_data.head())
except FileNotFoundError:
    print(f"Error: The file {clean_data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully.
               country_name  year  1_1_ffp_x1  1_2_ucdp  \
country_alpha3                                            
AFG             Afghanistan  2007         0.0       0.0   
ALB                 Albania  2007        0.44       1.0   
DZA                 Algeria  2007        0.43  0.458492   
AGO                  Angola  2007        0.24  0.899611   
ARG               Argentina  2007        0.72       1.0   

                1_3_wef_gci_subpillar:_5._security  1_4_political_terror  \
country_alpha3                                                             
AFG                                           <NA>                   0.0   
ALB                                       0.691667                  0.75   
DZA                                       0.768333                  0.25   
AGO                                           0.86                   0.5   
ARG                                           0.69                  0.75   

                1_5_ffp_p3 

/Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/src/idos_ppp/data_management/idos_datamanagement.py:144: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_dta = raw_dta.replace({"XXX": pd.NA, "YYY": pd.NA})
/Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/src/idos_ppp/data_management/idos_datamanagement.py:144: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_dta = raw_dta.replace({"XXX": pd.NA, "YYY": pd.NA})
/Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/src/idos_ppp/data_management/idos_datamanagement.py:144: FutureWarning: Downca

In [14]:
def are_csv_files_identical(file_path1, file_path2):
    """
    Check if two CSV files are identical.

    Parameters:
    - file_path1: str or Path, path to the first CSV file.
    - file_path2: str or Path, path to the second CSV file.

    Returns:
    - bool: True if the files are identical, False otherwise.
    """
    try:
        # Read the CSV files into DataFrames
        df1 = pd.read_csv(file_path1)
        df2 = pd.read_csv(file_path2)

        # Compare the DataFrames
        return df1.equals(df2)
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# Example usage
directory = Path(BLD / "data")  # Replace with your directory path
file1 = directory / "clean_data_new.csv"
file2 = directory / "clean_data.csv"

if are_csv_files_identical(file1, file2):
    print("The CSV files are identical.")
else:
    print("The CSV files are not identical.")


The CSV files are identical.


In [13]:
clean_data_2 = pd.read_csv(BLD / "data" / "clean_data.csv")
clean_data_2.describe

<bound method NDFrame.describe of     country_alpha3 country_name  year  1_1_ffp_x1  1_2_ucdp  \
0              AFG  Afghanistan  2007        0.00  0.000000   
1              ALB      Albania  2007        0.44  1.000000   
2              DZA      Algeria  2007        0.43  0.458492   
3              AGO       Angola  2007        0.24  0.899611   
4              ARG    Argentina  2007        0.72  1.000000   
..             ...          ...   ...         ...       ...   
725            URY      Uruguay  2019        0.77  1.000000   
726            VNM      Vietnam  2019        0.51  1.000000   
727            YEM  Yemen, Rep.  2019        0.00  0.000000   
728            ZMB       Zambia  2019        0.30  1.000000   
729            ZWE     Zimbabwe  2019        0.27  1.000000   

     1_3_wef_gci_subpillar:_5._security  1_4_political_terror  1_5_ffp_p3  \
0                                   NaN                  0.00        0.18   
1                              0.691667                

In [7]:
clean_data_2.iloc[:, 0]

0      AFG
1      ALB
2      DZA
3      AGO
4      ARG
      ... 
725    URY
726    VNM
727    YEM
728    ZMB
729    ZWE
Name: country_alpha3, Length: 730, dtype: string

In [17]:
clean_data_2

,country_alpha3,country_name,year,1_1_ffp_x1,1_2_ucdp,1_3_wef_gci_subpillar:_5._security,1_4_political_terror,1_5_ffp_p3,protection,2_1_1_wef_gcr_2nd_pillar:_infrastructure,...,2_6_1_share_of_wage_employment_on_work_age_pop,2_6_2_working_poverty_head_count_rate_(percentage_of_persons_living_in_poverty_in_spite_of_being_employed),2_7_1_wef_gci_subpillar:_1._property_rights,2_7_2_wef_gci_subpillars_2_and_3._ethics_and_corruption;_undue_influence,2_8_1_wef_gci_subpillar:_1._domestic_competition,2_8_2_wef_gci_subpillar:_4._government_efficiency,provision,3_1_v_dem_index_on_electoral_democracy,3_2_wb_voice_and_accountability_indicator,participation
0,AFG,Afghanistan,2007,0.00,0.000000,NaN,0.00,0.18,0.045000,NaN,...,0.129010,NaN,NaN,NaN,NaN,NaN,0.222057,0.395,0.309672,0.352336
1,ALB,Albania,2007,0.44,1.000000,0.691667,0.75,0.46,0.668333,0.292857,...,0.404818,0.9981,0.402857,0.331429,0.495714,0.412857,0.509244,0.544,0.535866,0.539933
2,DZA,Algeria,2007,0.43,0.458492,0.768333,0.25,0.26,0.430000,0.428571,...,0.689655,0.9981,0.557143,0.491429,0.575714,0.515714,0.545731,0.335,0.323880,0.329440
3,AGO,Angola,2007,0.24,0.899611,0.860000,0.50,0.25,0.500000,0.268571,...,0.337941,0.6801,0.395714,0.383571,0.490000,0.400000,0.346479,0.177,0.285006,0.231003
4,ARG,Argentina,2007,0.72,1.000000,0.690000,0.75,0.63,0.720000,0.432857,...,0.764526,0.9956,0.417143,0.289286,0.477143,0.360000,0.591369,0.797,0.600713,0.698856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,URY,Uruguay,2019,0.77,1.000000,0.712175,1.00,0.71,0.838435,0.686745,...,0.714549,0.9997,0.684227,0.700000,0.523626,0.579126,0.734595,0.890,0.756760,0.823380
726,VNM,Vietnam,2019,0.51,1.000000,0.772168,0.50,0.22,0.600434,0.659247,...,0.451281,0.9841,0.469491,0.330000,0.536697,0.506609,0.522454,0.216,0.237707,0.226853
727,YEM,"Yemen, Rep.",2019,0.00,0.000000,0.430356,0.00,0.01,0.088071,0.338541,...,0.440948,0.5292,0.258078,0.140000,0.383157,0.248502,0.233509,0.116,0.163662,0.139831
728,ZMB,Zambia,2019,0.30,1.000000,0.695670,0.50,0.26,0.551134,0.432737,...,0.245761,0.4608,0.398926,0.350000,0.456419,0.413051,0.359100,0.338,0.453870,0.395935


In [9]:
from idos_ppp.parameters import countries_to_leave_out
# Check if a particular country is in the "country_name" column
for country in countries_to_leave_out:
    if country in clean_data['country_name'].values:
        print(f"{country} is in the Country column.")
    else:
        print(f"{country} is not in the Country column.")

Belarus is not in the Country column.
Eswatini is not in the Country column.
Gabon is not in the Country column.
Iraq is not in the Country column.
Papua New Guinea is not in the Country column.
South Sudan is not in the Country column.
Sudan is not in the Country column.
Uzbekistan is not in the Country column.


In [10]:
len(countries_to_leave_out)

8

# PLAY WITH LISTS OF COUNTRIES

In [11]:
european_union = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland",
    "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland",
    "Portugal", "Romania", "Slovak Republic", "Slovenia", "Spain", "Sweden"
]
len(european_union)

27

In [36]:
# Conflic / Post-conflict (from Heydemann 2025)
conflict_countries = [
    "Libya", "Syrian Arab Republic", "Yemen, Rep."
]

conflict_and_postconflict_countries = [
    "Iraq", "Lebanon", "Libya", "Syrian Arab Republic", "Yemen, Rep."
]

# GCC countries (and repressive ones) (from Heydemann 2025)
gcc_high_income_countries = [
    "Bahrain", "Kuwait", "Oman", "Qatar", "Saudi Arabia", "United Arab Emirates"
]

repressive_countries = [
    "Egypt, Arab Rep.", "Jordan", "Morocco", "Tunisia"
] # Tunisia post-2020

gcc_and_repressive_countries = [
    "Bahrain", "Kuwait", "Oman", "Qatar", "Saudi Arabia", "United Arab Emirates", "Egypt, Arab Rep.", "Jordan", "Morocco", "Tunisia"
] # Tunisia post-2020

print(len(conflict_countries), len(conflict_and_postconflict), len(gcc_high_income_countries), len(repressive_countries), len(gcc_and_repressive_countries))

3 5 6 4 10


In [31]:
## Check Merged Dataset


# Define the file paths
pkl_file_path = BLD / "data" / "merged_data.pkl"
csv_file_path = BLD / "data" / "merged_data.csv"

# Load the .pkl file
df = pd.read_pickle(pkl_file_path)

# Save as CSV
df.to_csv(csv_file_path)

# Display the DataFrame
print(list(df["country_name"]))

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belgium', 'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Lithuania', 'Luxembourg', 'Mad

In [ ]:
def task_conflict_and_postconflict_countries(
    merged_data=BLD / "data" / "merged_data.pkl",
    produces=BLD / "analysis" / "conflict_and_postconflict_countries_data.pkl",
):
    """Task to create the corresponding dataset constrained to countries in the list conflict_countries."""
    data = pd.read_pickle(merged_data)
    reset_index_df = data.reset_index()
    conflict_and_postconflict_countries_df = reset_index_df[reset_index_df['country_name'].isin(conflict_and_postconflict_countries)]
    conflict_and_postconflict_countries_df = conflict_and_postconflict_countries_df.set_index(['country_alpha3', 'year'])
    conflict_and_postconflict_countries_df.to_pickle(produces)

In [39]:
# Lists of countries
country_lists = {
    "conflict_countries": conflict_countries,
    "conflict_and_postconflict_countries": conflict_and_postconflict_countries,
    "gcc_high_income_countries": gcc_high_income_countries,
    "repressive_countries": repressive_countries,
    "gcc_and_repressive_countries": gcc_and_repressive_countries
}

def process_and_save_country_list(country_list_name, country_list, pkl_file_path, output_dir):
    """
    Process the DataFrame for a given list of countries and save the filtered DataFrame as a CSV.

    Parameters:
    - country_list_name: str, the name of the country list for naming the output file.
    - country_list: list, the list of country names to filter the DataFrame.
    - pkl_file_path: Path, the path to the input .pkl file.
    - output_dir: Path, the directory where the output CSV file will be saved.
    """
    # Load the .pkl file
    df = pd.read_pickle(pkl_file_path)

    # Reset the index to access the 'country_name' column
    reset_index_df = df.reset_index()

    # Filter the DataFrame based on the country list
    filtered_df = reset_index_df[reset_index_df['country_name'].isin(country_list)]

    # Set the index back to 'country_alpha3' and 'year'
    filtered_df = filtered_df.set_index(['country_alpha3', 'year'])

    # Define the output file path
    output_csv_file_path = output_dir / f"{country_list_name}_data.csv"

    # Save the filtered DataFrame as a CSV
    filtered_df.to_csv(output_csv_file_path)
    print(f"Saved {country_list_name} data to {output_csv_file_path}")

# Define the file paths
pkl_file_path = BLD / "data" / "merged_data.pkl"
output_dir = BLD / "analysis"

# Process each country list
for list_name, country_list in country_lists.items():
    process_and_save_country_list(list_name, country_list, pkl_file_path, output_dir)


Saved conflict_countries data to /Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/bld/analysis/conflict_countries_data.csv
Saved conflict_and_postconflict_countries data to /Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/bld/analysis/conflict_and_postconflict_countries_data.csv
Saved gcc_high_income_countries data to /Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/bld/analysis/gcc_high_income_countries_data.csv
Saved repressive_countries data to /Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/bld/analysis/repressive_countries_data.csv
Saved gcc_and_repressive_countries data to /Users/sergeimolinari/Desktop/IDOS/IDOS-PPP/bld/analysis/gcc_and_repressive_countries_data.csv


In [ ]:
# Display the DataFrames
gcc_and_repressive_countries_data = pd.read_csv(output_dir / "gcc_and_repressive_countries_data.csv")
gcc_and_repressive_countries_data


,country_alpha3,year,country_name,continent,1_1_ffp_x1,1_2_ucdp,1_3_wef_gci_subpillar:_5._security,1_4_political_terror,1_5_ffp_p3,protection,...,2_6_1_share_of_wage_employment_on_work_age_pop,2_6_2_working_poverty_head_count_rate_(percentage_of_persons_living_in_poverty_in_spite_of_being_employed),2_7_1_wef_gci_subpillar:_1._property_rights,2_7_2_wef_gci_subpillars_2_and_3._ethics_and_corruption;_undue_influence,2_8_1_wef_gci_subpillar:_1._domestic_competition,2_8_2_wef_gci_subpillar:_4._government_efficiency,provision,3_1_v_dem_index_on_electoral_democracy,3_2_wb_voice_and_accountability_indicator,participation
0,BHR,2007,Bahrain,Asia,0.47,1.000000,0.906667,0.75,0.53,0.750000,...,0.973079,1.0000,0.748571,0.555714,0.654286,0.607143,0.561759,0.232,0.348533,0.290267
1,EGY,2007,"Egypt, Arab Rep.",Africa,0.24,1.000000,0.838333,0.25,0.15,0.250000,...,0.611409,0.9824,0.648571,0.530000,0.605714,0.515714,0.518716,0.213,0.288400,0.250700
2,JOR,2007,Jordan,Asia,0.31,1.000000,0.993333,0.50,0.38,0.500000,...,0.855741,0.9982,0.738571,0.590000,0.661429,0.590000,0.656579,0.238,0.383264,0.310632
3,KWT,2007,Kuwait,Asia,0.60,1.000000,0.955000,0.75,0.35,0.750000,...,0.983100,1.0000,0.687143,0.568571,0.628571,0.562857,0.550464,0.319,0.408689,0.363845
4,MAR,2007,Morocco,Africa,0.60,1.000000,0.843333,0.50,0.34,0.600000,...,0.469434,0.9861,0.628571,0.508571,0.618571,0.554286,0.535744,0.260,0.377332,0.318666
5,OMN,2007,Oman,Asia,0.80,1.000000,0.871667,1.00,0.34,0.871667,...,0.965143,1.0000,0.682857,0.682857,0.655714,0.670000,0.550540,0.158,0.308704,0.233352
6,QAT,2007,Qatar,Asia,0.54,1.000000,0.996667,0.75,0.53,0.750000,...,0.995067,1.0000,0.755714,0.749286,0.665714,0.687143,0.520143,0.082,0.313021,0.197510
7,SAU,2007,Saudi Arabia,Asia,0.35,1.000000,0.866667,0.25,0.12,0.350000,...,0.946858,1.0000,0.675714,0.597857,0.674286,0.598571,0.583610,0.018,0.184564,0.101282
8,TUN,2007,Tunisia,Africa,0.70,1.000000,0.943333,0.50,0.27,0.700000,...,0.721303,0.9935,0.742857,0.719286,0.710000,0.722857,0.643586,0.191,0.247598,0.219299
9,ARE,2007,United Arab Emirates,Asia,0.60,1.000000,0.996667,0.75,0.39,0.750000,...,0.967118,1.0000,0.724286,0.728571,0.697143,0.711429,0.561666,0.068,0.338804,0.203402
